In [11]:
import os
import sys

import numpy as np

from data.load_data import get_plant_leaves_dataset

In [12]:
X_train, X_validation, X_test, y_train, y_validation, y_test = get_plant_leaves_dataset()

In [13]:
# Image Data Shape
print(X_train.shape)
print(X_validation.shape)
print(X_test.shape)
# Label Data Shape
print(y_train.shape)
print(y_validation.shape)
print(y_test.shape)

(3601, 256, 256, 3)
(450, 256, 256, 3)
(451, 256, 256, 3)
(3601, 2)
(450, 2)
(451, 2)


### Model
The model is of a ResNet architecture, with convolution layers and the input from each layer being added to the end of it. Rather than fully connected layers, Global Average Pooling is used. The model is then trained on 100 epochs.

from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAvgPool2D
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, ReLU, Softmax
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAvgPool2D
from tensorflow.keras.layers import Input, Activation, Dense, Dropout, ReLU, Softmax
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [15]:
# ResNet Model
def create_model(input, classes, l2_reg = 0.005):
   reg = l2(l2_reg)

   # Model
   img_input = Input(input)
   model = Conv2D(4, kernel_size = (3, 3), strides = (1, 1), kernel_regularizer = reg, use_bias = False)(img_input)
   model = BatchNormalization()(model)
   model = ReLU()(model)
   model = Conv2D(4, kernel_size = (3, 3), strides = (1, 1), kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = ReLU()(model)

   res = Conv2D(8, kernel_size = (1, 1), strides = (2, 2), kernel_regularizer = reg, use_bias = False)(model)
   res = BatchNormalization()(res)

   model = Conv2D(8, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = ReLU()(model)
   model = Conv2D(8, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(model)
   model = layers.add([model, res])

   res = Conv2D(16, kernel_size = (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(model)
   res = BatchNormalization()(res)

   model = Conv2D(16, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = ReLU()(model)
   model = Conv2D(16, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(model)
   model = layers.add([model, res])

   res = Conv2D(32, kernel_size = (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(model)
   res = BatchNormalization()(res)

   model = Conv2D(32, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = ReLU()(model)
   model = Conv2D(32, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(model)
   model = layers.add([model, res])

   res = Conv2D(64, kernel_size = (1, 1), strides = (2, 2), padding = 'same', use_bias = False)(model)
   res = BatchNormalization()(res)

   model = Conv2D(64, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = ReLU()(model)
   model = Conv2D(64, kernel_size = (3, 3), padding = 'same', kernel_regularizer = reg, use_bias = False)(model)
   model = BatchNormalization()(model)
   model = MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'same')(model)
   model = layers.add([model, res])

   model = Conv2D(classes, kernel_size = (3, 3), padding = 'same')(model)
   model = GlobalAvgPool2D()(model)

   output = Softmax(name = 'predictions')(model)

   model = Model(img_input, output)
   return model

In [16]:
model = create_model((256, 256, 3), 2)
model.compile(
   optimizer = Adam(),
   loss = categorical_crossentropy,
   metrics = ['accuracy']
)

In [17]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 254, 254, 4)  108         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 254, 254, 4)  16          conv2d_15[0][0]                  
__________________________________________________________________________________________________
re_lu_6 (ReLU)                  (None, 254, 254, 4)  0           batch_normalization_14[0][0]     
_______________________________________________________________________________________

In [18]:
datagen = ImageDataGenerator(horizontal_flip = True)

train_flow = datagen.flow(X_train, y_train, 32)
validation_flow = datagen.flow(X_validation, y_validation)

In [19]:
# Callbacks
datadir = os.path.join(os.getcwd(), "data", "model")
early_stop = EarlyStopping(monitor = 'val_loss', patience = 50)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = int(50 / 4), verbose = 1)
save_path = os.path.join(datadir, "model", "Model-{epoch:02d}-{val_accuracy:.4f}.hdf5")
checkpoint = ModelCheckpoint(save_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
callbacks = [checkpoint, reduce_lr, early_stop]

In [8]:
model.fit_generator(
   train_flow,
   epochs = 100,
   verbose = 1,
   callbacks = callbacks,
   validation_data = validation_flow
)